In [3]:
# !CUDA_LAUNCH_BLOCKING=1 python BERT+CRF.ipynb


In [1]:
!pip install torchcrf
!pip install transformers datasets seqeval
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers import BertTokenizerFast
from datasets import load_dataset
import torch
from torch import nn
from transformers import BertModel
from TorchCRF import CRF
from transformers import AdamW, get_scheduler
from torch.utils.data import DataLoader
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
import evaluate


2025-03-09 12:02:09.734993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741521729.747460    2621 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741521729.751210    2621 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:


def create_dataset(json_path):
    """
    Reads the JSON file with token and label data, flattens it,
    converts labels to numeric IDs, and splits the data into train/test sets.
    
    Returns:
        DatasetDict: A Hugging Face DatasetDict with 'train' and 'test' splits.
    """
    js = pd.read_json(json_path, encoding="utf-8")
    tokens_flat = []
    labels_flat = []
    sentence_ids_flat = []
    
    # Flatten the lists
    for _, row in js.iterrows():
        tokens = row["tokens"]
        labels = row["labels"]
        sentence_id = row["sentence_id"]  # This is an integer
    
        # Ensure token-label length matches before extending
        if len(tokens) == len(labels):
            tokens_flat.extend(tokens)  
            labels_flat.extend(labels)  
            sentence_ids_flat.extend([sentence_id] * len(tokens))  # Repeat sentence_id for each token
        else:
            print(f"Skipping sentence_id {sentence_id} due to mismatched lengths: {len(tokens)} tokens vs {len(labels)} labels")
    
    # Ensure final lengths match
    assert len(tokens_flat) == len(labels_flat) == len(sentence_ids_flat), "Mismatch in list lengths!"
    
    # Convert labels to numeric IDs
    unique_labels = list(set(labels_flat))

    global id2label
    id2label = {idx: label for idx, label in enumerate(unique_labels)}     
    global label2id
    label2id = {label: idx for idx, label in enumerate(unique_labels)}

    labels_numeric_flat = [label2id[label] for label in labels_flat]
    
    # Create Dataset dictionary
    dataset_dict = {
        "tokens": tokens_flat,
        "ner_tags": labels_flat,
        "sentence_id": sentence_ids_flat,
        "labels_numeric": labels_numeric_flat
    }
    
    # Convert to Hugging Face Dataset
    dataset = Dataset.from_dict(dataset_dict)
    
    # Split into train and test sets
    dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
    
    # Store in DatasetDict format
    dataset_final = DatasetDict({
        "train": dataset_split["train"],
        "test": dataset_split["test"]
    })
    
    # Display a sample
    print(dataset_final["train"][0])
    print(f"Training set size: {len(dataset_final['train'])}")
    print(f"Test set size: {len(dataset_final['test'])}")

    return dataset_final

In [7]:

model_name= "dbmdz/bert-large-cased-finetuned-conll03-english"
# model_name= "ProsusAI/finbert"

tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset= create_dataset('Annoted_JSON_090225.json')
token = tokenizer(dataset['train']['tokens'][:5], is_split_into_words=True, padding=True, truncation=True)

print(f"Tokens: {token.word_ids()}")

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

{'tokens': 'to', 'ner_tags': 'O', 'sentence_id': 1, 'labels_numeric': 6}
Training set size: 393
Test set size: 99
Tokens: [None, 0, 1, 2, 3, 4, 4, None]


In [8]:

# class BertCRFNER(nn.Module):
#     def __init__(self, bert_model_name, num_labels):
#         super().__init__()
#         self.bert = BertModel.from_pretrained(bert_model_name)
#         self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
    
#     def forward(self, input_ids, attention_mask=None, labels=None):
#         outputs = self.bert(input_ids, attention_mask=attention_mask)
#         logits = self.classifier(outputs.last_hidden_state)
        
#         if labels is not None:
#             loss_fn = nn.CrossEntropyLoss()
#             loss = loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
#             return {"loss": loss, "logits": logits}  # Ensure loss is returned
        
#         return {"logits": logits}  # No loss during inference

# # Load the model
# model_name = "bert-base-cased"
# num_labels = len(set(dataset["train"]["ner_tags"]))  
# model = BertCRFNER(model_name, num_labels)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

In [9]:


# Load a dataset (can be replaced with a custom dataset)
# dataset = create_dataset("Annoted_JSON_090225.json")

# Load a BERT tokenizer


# Tokenization function
def tokenize_fn(batch):
    """
    Tokenizes the batch of tokens and aligns their labels.
    
    Args:
        batch (dict): Batch with keys "tokens" and "labels_numeric".
        
    Returns:
        dict: Tokenized inputs with aligned labels.
    """

    def align_target(labels, word_ids):
        """
        Aligns the labels with the tokenized word IDs.
        
        Args:
            labels (list): Original labels for a sentence.
            word_ids (list): Word IDs from tokenization.
        
        Returns:
            list: Aligned labels (with -100 for special tokens).
        """
        begin2inside = {1: 2, 3: 4, 5: 6, 7: 8}
        align_labels = []
        last_word = None
        for word in word_ids:
            if word is None:
                label = -100  # For special tokens like [CLS] and [SEP]
            elif word != last_word:
                label = labels[word]
            else:
                label = labels[word]
            align_labels.append(label)
            last_word = word
        return align_labels
    
    tokenized_inputs = tokenizer(
        batch["tokens"],
        truncation=True,
        padding=True,
        is_split_into_words=True,
        max_length=512,
        return_tensors="np"
    )

    # Align labels
    labels_batch = batch["labels_numeric"]

    print(f"Labels Numeric: {labels_batch}")

    aligned_targets_batch = []
    
    aligned_targets_batch.append(align_target(labels_batch, tokenized_inputs.word_ids()))

    # Add the aligned labels to the tokenized inputs
    tokenized_inputs["labels"] = aligned_targets_batch
    print(f"Tokenized Labels: {tokenized_inputs}")
    return tokenized_inputs
    

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_fn, batched=True,  remove_columns=dataset['train'].column_names)

# Set the dataset format to PyTorch tensors
# tokenized_datasets.set_format("torch")

Map:   0%|          | 0/393 [00:00<?, ? examples/s]

Labels Numeric: [6, 6, 6, 9, 8, 19, 17, 14, 6, 6, 9, 6, 6, 3, 9, 19, 8, 6, 13, 6, 19, 6, 6, 6, 24, 6, 17, 13, 3, 6, 6, 6, 5, 8, 6, 28, 6, 6, 17, 30, 19, 6, 6, 6, 11, 6, 23, 6, 19, 5, 6, 6, 30, 6, 6, 6, 6, 18, 14, 7, 28, 13, 28, 7, 6, 1, 30, 6, 6, 6, 6, 6, 6, 6, 6, 11, 7, 28, 0, 19, 28, 6, 6, 6, 6, 6, 6, 28, 17, 28, 6, 6, 6, 6, 6, 6, 6, 6, 6, 23, 6, 2, 6, 19, 6, 6, 6, 6, 19, 6, 6, 10, 19, 6, 17, 6, 6, 6, 28, 6, 30, 18, 6, 30, 6, 30, 6, 1, 6, 6, 6, 6, 24, 6, 6, 6, 30, 6, 6, 6, 30, 24, 6, 19, 6, 19, 21, 6, 19, 2, 6, 17, 6, 6, 6, 9, 6, 21, 16, 6, 6, 6, 7, 6, 30, 6, 30, 6, 6, 6, 14, 24, 28, 6, 6, 7, 6, 19, 6, 6, 6, 30, 6, 6, 6, 6, 6, 6, 12, 6, 6, 7, 7, 30, 19, 6, 6, 6, 19, 6, 19, 6, 30, 6, 12, 19, 6, 6, 6, 6, 6, 22, 6, 30, 5, 24, 9, 3, 6, 6, 6, 7, 6, 6, 27, 6, 6, 6, 16, 6, 6, 6, 1, 6, 6, 6, 3, 9, 6, 8, 30, 3, 6, 30, 6, 6, 6, 28, 6, 6, 6, 6, 6, 21, 6, 6, 6, 6, 6, 6, 28, 6, 7, 17, 0, 6, 6, 6, 6, 12, 19, 10, 6, 6, 25, 6, 6, 6, 6, 6, 26, 11, 6, 6, 6, 6, 6, 30, 6, 6, 6, 20, 19, 6, 30, 5, 6, 6, 1

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Labels Numeric: [20, 19, 13, 6, 6, 6, 6, 16, 6, 6, 6, 6, 6, 30, 6, 11, 6, 6, 13, 6, 20, 15, 28, 6, 6, 6, 1, 6, 6, 6, 6, 6, 10, 16, 6, 6, 6, 28, 10, 30, 6, 6, 6, 15, 6, 28, 6, 19, 6, 6, 6, 6, 6, 6, 6, 6, 25, 7, 6, 6, 6, 5, 6, 6, 6, 6, 30, 6, 20, 6, 6, 6, 12, 30, 6, 19, 27, 1, 6, 6, 6, 6, 6, 8, 6, 19, 6, 6, 7, 6, 8, 6, 6, 6, 2, 21, 20, 6, 28]
Tokenized Labels: {'input_ids': array([[  101,   122,   117,  7393,   119,   126,  1703, 24824,  1105,
         2075,  1104,  1977,  1550,  1214,  1214,   119,  1110,  1106,
         1634,  7991,  1104,  1550,   114,   117,  2426, 12087, 24824,
         1109, 14691,   119,   122,  1542,   117,  3102,  1580,  1479,
          119,  5787,  1703,  2500,  1110,  1382,  5671,   114,  1118,
         1109,  1114,  3254, 17482,  5382,  7270,  2069,  1550,  1103,
         1108,  1447,   836,  1475,   117,  5599,  1604,  7270,  2069,
         1447,  1338,  1103,  1353,   117,   836, 22639,   117, 21606,
         1205,  4062,   117,  4372,  1104,  4434,  1107, 

In [10]:
tokenized_datasets
len(dataset['train']['ner_tags'])

393

In [11]:
label_names= list(dataset['train']['ner_tags'])
label_names= sorted(label_names)

In [12]:
def compute_metrics(logits_and_labels):
    ''' This is a simple function which will evaluate the model's output.'''

    logits, labels = logits_and_labels 

     # Load the seqeval metric which can evaluate NER and other sequence tasks
    metric = evaluate.load("seqeval")
    
    # Example usage: compute metric on a sample predictions and reference list 
    # predictions and references should be a list of lists containing predicted and true token labels
    
    # List of List Input  
    metric.compute(predictions = [['O' , 'B-ORG' , 'I-ORG']], 
                   references = [['O' , 'B-MISC' , 'I-ORG']])
    
    # Convert labels from set to list (if needed)
    if isinstance(labels, set):
        labels = list(labels)

    # Get predictions from the logits
    predictions = np.argmax(logits, axis=-1)
    
    # Remove ignored index (special tokens)
    str_labels = [
        [label_names[t] for t in label if t != -100] for label in labels
    ]
    
    str_preds = [
        [label_names[p] for (p, t) in zip(prediction, label) if t != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    # Compute metrics
    results = metric.compute(predictions=str_preds, references=str_labels)
    
    # Extract key metrics
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"], 
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]  
    }


In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
model = AutoModelForTokenClassification.from_pretrained(model_name, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

# Move model to GPU if available
device = "cpu"
model.to(device)

training_args = TrainingArguments(
  # Location to save fine-tuned model 
  output_dir = "fine_tuned_model",

  # Evaluate each epoch
  evaluation_strategy = "epoch",
    remove_unused_columns=False,

  # Learning rate for Adam optimizer
  learning_rate = 0.001, 
  
  # Batch sizes for training and evaluation
  per_device_train_batch_size = 8,
  per_device_eval_batch_size = 8,

  # Number of training epochs
  num_train_epochs = 3,

  # L2 weight decay regularization
  weight_decay = 0.01,
  
   no_cuda=True
)

# Train the model using the Trainer
trainer = Trainer(
  model=model,  # Use the custom BertCRFNER model
  args=training_args,
  train_dataset=tokenized_datasets['train'],
  eval_dataset=tokenized_datasets['test'],
  tokenizer=tokenizer,
  compute_metrics=compute_metrics,
  data_collator=data_collator,
)

trainer.train()

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([31]) in the model instanti

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,8.186843,0.860927,0.860927,0.860927,0.860927
2,No log,4.133596,0.860927,0.860927,0.860927,0.860927
3,No log,3.096928,0.860927,0.860927,0.860927,0.860927


/home/jovyan/.local/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=3, training_loss=5.214365005493164, metrics={'train_runtime': 10.7846, 'train_samples_per_second': 0.278, 'train_steps_per_second': 0.278, 'total_flos': 2786394921984.0, 'train_loss': 5.214365005493164, 'epoch': 3.0})

In [1]:
# from new_bert-crf import NERDataset, TokenizerAligner, NERTrainer

In [3]:
import json
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset
import torch
from torch import nn
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    BertTokenizerFast,
    AdamW,
    get_scheduler,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)

# Global label mappings will be set by the dataset loader
id2label = {}
label2id = {}

class NERDataset:
    def __init__(self, json_path: str):
        self.json_path = json_path
        self.dataset = None

    def create_dataset(self) -> DatasetDict:
        js = pd.read_json(self.json_path, encoding="utf-8")
        tokens_flat = []
        labels_flat = []
        sentence_ids_flat = []

        for _, row in js.iterrows():
            tokens = row["tokens"]
            labels = row["labels"]
            sentence_id = row["sentence_id"]  # This is an integer
            if len(tokens) == len(labels):
                tokens_flat.extend(tokens)
                labels_flat.extend(labels)
                sentence_ids_flat.extend([sentence_id] * len(tokens))
            else:
                print(f"Skipping sentence_id {sentence_id} due to mismatched lengths: {len(tokens)} tokens vs {len(labels)} labels")

        assert len(tokens_flat) == len(labels_flat) == len(sentence_ids_flat), "Mismatch in list lengths!"

        global id2label, label2id
        unique_labels = list(set(labels_flat))
        id2label = {idx: label for idx, label in enumerate(unique_labels)}
        label2id = {label: idx for idx, label in enumerate(unique_labels)}
        labels_numeric_flat = [label2id[label] for label in labels_flat]

        dataset_dict = {
            "tokens": tokens_flat,
            "ner_tags": labels_flat,
            "sentence_id": sentence_ids_flat,
            "labels_numeric": labels_numeric_flat
        }
        dataset = Dataset.from_dict(dataset_dict)
        dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
        self.dataset = DatasetDict({
            "train": dataset_split["train"],
            "test": dataset_split["test"]
        })

        print("Sample training data:", self.dataset["train"][0])
        print(f"Training set size: {len(self.dataset['train'])}")
        print(f"Test set size: {len(self.dataset['test'])}")
        return self.dataset


class TokenizerAligner:
    def __init__(self, model_name: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize_and_align_labels(self, dataset: DatasetDict) -> DatasetDict:
        def tokenize_fn(batch):
            def align_target(labels, word_ids):
                align_labels = []
                for word in word_ids:
                    if word is None:
                        label = -100  # For special tokens like [CLS] and [SEP]
                    else:
                        label = labels[word]
                    align_labels.append(label)
                return align_labels

            tokenized_inputs = self.tokenizer(
                batch["tokens"],
                truncation=True,
                padding=True,
                is_split_into_words=True,
                max_length=512,
                return_tensors="np"
            )
            labels_batch = batch["labels_numeric"]
            aligned_targets_batch = [align_target(labels_batch, tokenized_inputs.word_ids())]
            tokenized_inputs["labels"] = aligned_targets_batch
            return tokenized_inputs

        tokenized_datasets = dataset.map(tokenize_fn, batched=True, remove_columns=dataset['train'].column_names)
        tokenized_datasets.set_format("torch")
        print("Tokenization and alignment completed.")
        return tokenized_datasets


def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs["logits"]
    loss_fn = nn.CrossEntropyLoss()
    loss = loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
    return (loss, outputs) if return_outputs else loss


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)  # Get the predicted labels
    accuracy = (predictions == labels).mean()  # Basic accuracy
    return {"accuracy": accuracy}


class NERTrainer:
    def __init__(self, model_name: str, tokenizer, tokenized_datasets: DatasetDict):
        self.model_name = model_name
        self.tokenizer = tokenizer
        self.tokenized_datasets = tokenized_datasets
        self.data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
        self.device = "cpu"  # Change to "cuda" if GPU is available

        self.model = AutoModelForTokenClassification.from_pretrained(
            model_name,
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True
        )
        self.model.to(self.device)

    def train(self):
        training_args = TrainingArguments(
            output_dir="fine_tuned_model",
            evaluation_strategy="epoch",
            remove_unused_columns=False,
            learning_rate=0.001,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=3,
            weight_decay=0.01,
            no_cuda=True  # Set to False if using a GPU
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.tokenized_datasets['train'],
            eval_dataset=self.tokenized_datasets['test'],
            tokenizer=self.tokenizer,
            compute_metrics=compute_metrics,
            data_collator=self.data_collator,
        )

        trainer.train()
        return trainer


def log_results_to_excel(results, excel_path="model_results.xlsx"):
    # Check if the Excel file exists
    try:
        df = pd.read_excel(excel_path)
    except FileNotFoundError:
        # If the file doesn't exist, create a new one
        df = pd.DataFrame(columns=["Model", "Accuracy"])

    # Create a new DataFrame from the results to concatenate
    new_data = pd.DataFrame(results, columns=["Model", "Accuracy"])

    # Concatenate the new data with the existing DataFrame
    df = pd.concat([df, new_data], ignore_index=True)

    # Save to Excel
    df.to_excel(excel_path, index=False)
    print(f"Results logged to {excel_path}")

def main():
    # Define the list of models
    model_names = [
        "dbmdz/bert-large-cased-finetuned-conll03-english",
        "bert-base-cased",
        "bert-large-uncased"
    ]
    
    # Path to the JSON file
    json_path = "Annoted_JSON_090225.json"

    # Create dataset
    dataset_loader = NERDataset(json_path)
    dataset = dataset_loader.create_dataset()

    # Tokenize and align labels
    tokenizer_aligner = TokenizerAligner(model_names[0])  # Initialize with the first model
    tokenized_datasets = tokenizer_aligner.tokenize_and_align_labels(dataset)

    # Initialize an empty list to store results
    results = []

    for model_name in model_names:
        print(f"Training model: {model_name}")
        
        # Reinitialize the tokenizer and align the labels for each model
        tokenizer_aligner = TokenizerAligner(model_name)
        tokenized_datasets = tokenizer_aligner.tokenize_and_align_labels(dataset)
        
        # Train the model
        ner_trainer = NERTrainer(model_name, tokenizer_aligner.tokenizer, tokenized_datasets)
        trainer = ner_trainer.train()

        # Get the accuracy and store the result
        accuracy = trainer.evaluate()["eval_accuracy"]
        results.append((model_name, accuracy))

    # Log the results to an Excel file
    log_results_to_excel(results)


# if __name__ == "__main__":
main()


Sample training data: {'tokens': 'to', 'ner_tags': 'O', 'sentence_id': 1, 'labels_numeric': 13}
Training set size: 393
Test set size: 99


Map:   0%|          | 0/393 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Tokenization and alignment completed.
Training model: dbmdz/bert-large-cased-finetuned-conll03-english


Map:   0%|          | 0/393 [00:00<?, ? examples/s]

...

Tokenization and alignment completed.


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/jovyan/.local/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/jovyan/.local/lib/python3.11/site-packages/transformers/training_args.py:1609: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/tmp/ipykernel_2621/3791639246.py:150: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,8.859374,0.000000
2,No log,3.747200,0.013072
3,No log,2.658297,0.464052


Results logged to model_results.xlsx


/tmp/ipykernel_2621/3791639246.py:176: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data], ignore_index=True)


Found existing installation: wandb 0.19.8
Uninstalling wandb-0.19.8:
  Would remove:
    /home/jovyan/.local/bin/wandb
    /home/jovyan/.local/bin/wb
    /home/jovyan/.local/lib/python3.11/site-packages/package_readme.md
    /home/jovyan/.local/lib/python3.11/site-packages/wandb-0.19.8.dist-info/*
    /home/jovyan/.local/lib/python3.11/site-packages/wandb/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
